In [2]:
%pip install torch --quiet
%pip install torchprofile --quiet

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 72.4 MB/s eta 0:00:00ta 0:00:01


In [1]:
import torch
from torch import nn
import os
from torchprofile import profile_macs

In [2]:
src = torch.randn((10, 32, 5))  # (sequence_length, batch_size, input_dim)
tgt = torch.randn((20, 32, 5))

In [3]:
src.shape

torch.Size([10, 32, 5])

In [4]:
src.size()[-1]

5

In [9]:
"""doc
"""

import torch
from torch import nn


class TokenEmbedding(nn.Module):
    def __init__(self, number_vocab=1000, max_len=100, number_hidden=64):
        super().__init__()
        self.postional_embedding_layers = nn.Embedding(number_vocab, number_hidden)
        self.embedding_layers = nn.Embedding(max_len, number_hidden)

    def forward(self, input_x):
        max_len = input_x.size()[-1]
        input_x = self.embedding_layers(input_x)
        # Generate positions using torch.arange
        positions = torch.arange(0, max_len)
        positions = self.postional_embedding_layers(positions)
        return input_x + positions


class LandmarkEmbedding(nn.Module):
    def __init__(self, input_dim = None, number_hidden=64, max_len=100):
        super().__init__()
        self.conv1 = nn.Conv1d(
            in_channels=input_dim,
            out_channels=number_hidden,
            kernel_size=11,
            padding="same",
            stride=1,
        )
        self.conv2 = nn.Conv1d(
            in_channels=number_hidden,
            out_channels=number_hidden,
            kernel_size=11,
            padding="same",
            stride=1,
        )
        self.conv3 = nn.Conv1d(
            in_channels=number_hidden,
            out_channels=number_hidden,
            kernel_size=11,
            padding="same",
            stride=1,
        )
        self.postions_embedding_layers = nn.Embedding(max_len, number_hidden)
        self.seq_nn = nn.Sequential(
            self.conv1, nn.ReLU(), self.conv2, nn.ReLU(), self.conv3, nn.ReLU()
        )

    def forward(self, input_x):
        outputs = self.seq_nn(input_x)
        return outputs


class Transformer(nn.Module):
    def __init__(
        self,
        input_dim,
        output_dim,
        source_maxlen=100,
        target_maxlen=100,
        no_multi_heads=6,
    ):
        super().__init__()
        num_encoder_layers = num_decoder_layers = 6
        encoder_forward_dim = 100
        # Define encoder and decoder layers
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim,
            nhead=no_multi_heads,
            dim_feedforward=encoder_forward_dim,
            activation="relu",
        )

        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model=input_dim,
            nhead=no_multi_heads,
            dim_feedforward=output_dim,
            activation="relu",
        )

        # Define encoder and decoder
        self.transformer_encoder = nn.TransformerEncoder(
            self.encoder_layer, num_layers=num_encoder_layers
        )
        self.transformer_decoder = nn.TransformerDecoder(
            self.decoder_layer, num_layers=num_decoder_layers
        )

        # Input and output linear layers
        self.input_linear = LandmarkEmbedding(input_dim=input_dim,max_len=source_maxlen)
        self.target_linear = TokenEmbedding(max_len=target_maxlen)
        self.num_classes = 60
        self.output_linear = nn.Linear(output_dim, self.num_classes)

    def forward(self, input_x, input_y):
        # Apply EMbedding
        input_x = self.input_linear(input_x)

        # Transformer encoding
        memory = self.transformer_encoder(input_x)

        # Apply linear layer to the target
        input_y = self.target_linear(input_y)

        # Transformer decoding
        output = self.transformer_decoder(input_y, memory)

        # Apply linear layer to the output
        output = self.output_linear(output)

        return output

    # TODO code generate for inference
    def generate(
        self,
    ):
        pass


In [17]:
#RuntimeError: Given groups=1, weight of size [11, 64, 2], 
#expected input[100, 32, 513] to have 64 channels, but got 32 channels instead
# Example usage:
input_dim = 513  # Adjust based on your input dimension
output_dim = 256  # Adjust based on your output dimension
nhead = 9
batch_size = 16
sequnce = 100
# Instantiate the model
model = Transformer(input_dim, output_dim,no_multi_heads=nhead)

# Create dummy input
src = torch.randn((batch_size, input_dim, input_dim))  # (sequence_length, batch_size, input_dim)
tgt = torch.randn((batch_size, input_dim, input_dim)).long()  # (sequence_length, batch_size, input_dim)

# Forward pass
output = model(src, tgt)

# Print the output shape
print("Output shape:", output.shape)

/workspace/.pyenv_mirror/user/current/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


IndexError: index out of range in self

In [ ]:
input_dim = 513
num_heads = 9

# Check if embed_dim is divisible by num_heads
if input_dim % num_heads != 0:
    print("Error: embed_dim must be divisible by num_heads")
else:
    print("No error, the configuration is valid.")


No error, the configuration is valid.


In [16]:
positions = torch.arange(0, 100)
positions

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 91, 92, 93, 94, 95, 96, 97, 98, 99])